In [2]:
import cv2
import numpy as np
import math

USE_LINE_INSECT = 3 # [昆虫画像256/line切り替え設定] 2: AIの線画 0: 256昆虫画像を使用する 1: line昆虫画像を使用する
DISP_DOG_WINDOW = 0 # [犬画像の表示設定] 0:最終画像以外の表示なし 1: 初期ロード直後の画像も表示 2: さらに、輪郭抽出時の加工画像等も表示
BUNKATSU = 4 #中間層の分割数

#
# モード設定変数(輪郭配置関連)
#
PLACE_EDGE_INSECTS = 1 # 0:輪郭線沿いに昆虫を表示しない 1:輪郭線沿いに昆虫を表示する
DRAW_EDGE_FIGURE = 1 # 1:昆虫画像位置ガイドとしての直線・楕円を描画する  0:昆虫画像位置ガイドとしての直線・楕円を描画しない
DISP_EDGE_INSECT_WINDOW = 0 # [別ウィンドウでの昆虫の表示] 0:表示なし 1: アフィン変換後の画像を表示 2: さらに、初期ロード直後の画像も表示

GREEN = (0, 255, 0)
THCK = 1

insect_images_AI_c_new = [
    'mantis10','mantis11','mantis12','mantis13','mantis14','mantis15','mantis16','mantis17','mantis18','mantis19',
    'mantis110','mantis111','mantis112','mantis113','mantis114','mantis115','mantis116','mantis117','mantis118','mantis119',
    'mantis120','mantis121','mantis122','mantis123','mantis124','mantis125','mantis126','mantis127',
]


def calc_midpoint(start_p, end_p):
    return (start_p[0] + end_p[0]) // 2, (start_p[1] + end_p[1]) // 2

def calc_distance(start_p, end_p):
    delta_x, delta_y = calc_delta(start_p, end_p)
    return int(math.sqrt(delta_x**2 + delta_y**2))

def calc_delta(start_p, end_p):
    delta_x = end_p[0] - start_p[0]
    delta_y = end_p[1] - start_p[1]

    return delta_x, delta_y

def calc_angle(start_p, end_p):
    delta_x, delta_y = calc_delta(start_p, end_p)

    angle_radians = math.atan2(delta_y, delta_x)
    angle_degrees = math.degrees(angle_radians)

    return (angle_degrees + 90) % 360

'''def calculate_angle(point1, point2):
    """2点間の角度を計算"""
    dx = point2[0] - point1[0]
    dy = point2[1] - point1[1]
    return math.degrees(math.atan2(dy, dx))
'''
# 線分間の角度を計算する関数
def calculate_angle(vec1, vec2):
    # ベクトルの内積と長さを計算
    dot_product = vec1[0] * vec2[0] + vec1[1] * vec2[1]
    mag_vec1 = math.sqrt(vec1[0]**2 + vec1[1]**2)  # ベクトル1の大きさ
    mag_vec2 = math.sqrt(vec2[0]**2 + vec2[1]**2)  # ベクトル2の大きさ

    # 角度（ラジアン）を計算し、度数に変換
    angle_rad = math.acos(dot_product / (mag_vec1 * mag_vec2))
    angle_deg = math.degrees(angle_rad)
    angle_deg = math.floor(angle_deg)
    return angle_deg

def get_largest_contour(img):

    # 画像をグレースケールに変換
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 画像を二値化
    ret, img_bin = cv2.threshold(img_gray, 127, 255,0)

    # 画像輪郭抽出
    contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #   
    # rint('num of contours=' + str(len(contours)))

    # 最大輪郭決定
    largest_contour = []
    num_points = 0
    for cnt in contours:
        if len(cnt) > num_points:
            num_points = len(cnt)
            largest_contour = cnt

    return largest_contour

def split_points(start_point, end_point):
    # 1/2 と 1/2 の位置で分割
    mid_point1 = [start_point[0] + (end_point[0] - start_point[0]) / 2, 
                  start_point[1] + (end_point[1] - start_point[1]) / 2]
    
    mid_point2 = [start_point[0] + 2 * (end_point[0] - start_point[0]) / 2, 
                  start_point[1] + 2 * (end_point[1] - start_point[1]) / 2]
    
    return mid_point1, mid_point2

def calc_angle_from_points(start_point, end_point):
    # 座標の差を求める
    dx = end_point[0] - start_point[0]
    dy = end_point[1] - start_point[1]
    
    # atan2関数を使って角度を計算（ラジアン）
    angle_rad = math.atan2(dy, dx)
    
    # ラジアンを度に変換
    angle_deg = math.degrees(angle_rad)
    
    #小数点以下切り捨て
    angle_deg_t = math.floor(angle_deg)
    
    return angle_deg_t

def rotate_image(img, angle, center=None, scale=1.0):
    (h, w) = img.shape[:2]
    
    if center is None:
        center = (w // 2, h // 2)
        
    # 回転行列を生成
    rot_mat = cv2.getRotationMatrix2D(center, angle, scale)
    rotated_img = cv2.warpAffine(img, rot_mat, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))
    
    return rotated_img

def process_image(img_insect, img_back, midpoint_x, midpoint_y):

    insect_height = img_insect.shape[0]
    insect_width = img_insect.shape[1]

    backimg_height = img_back.shape[0]
    backimg_width = img_back.shape[1]

    # (1) 背景画像指定点(楕円またはパーツ中心点)と、昆虫画像の中心点が一致する形となるよう配置位置を決定する
    # (1)-1
    ###################################################半分
    y_reduce_upper = 0
    y_offset = midpoint_y - insect_height // 2
    if (y_offset < 0):
        y_reduce_upper = -y_offset
        y_offset = 0

    # (1)-2
    x_reduce_left = 0
    x_offset = midpoint_x - insect_width // 2
    ###################################################半分
    if (x_offset < 0):
        x_reduce_left = -x_offset
        x_offset = 0

    # (2) 背景画像合成位置(左上、右下座標)の決定
    # (1-1)あるいは(1-2)において、昆虫画像位置がマイナスとなり0に補正した場合は、その分、右側、あるいは下側の座標値を減らし調整する
    y1, y2 = y_offset, y_offset + insect_height - y_reduce_upper
    x1, x2 = x_offset, x_offset + insect_width - x_reduce_left
    # (3) Length調整
    # 昆虫画像がy方向下側にはみ出る場合は、はみ出る長さをy_reduce_lower変数に格納し、また背景画像合成位置下側(y2)をはみ出る分だけ減算する
    y_reduce_lower = 0
    if (y2 > backimg_height):
        y_reduce_lower = y2 - backimg_height
        y2 -= y_reduce_lower

    # 昆虫画像がx方向右側にはみ出る場合は、はみ出る長さをx_reduce_right変数に格納し、また背景画像合成位置右側(x2)をはみ出る分だけ減算する
    x_reduce_right = 0
    if (x2 > backimg_width):
        x_reduce_right = x2 - backimg_width
        x2 -= x_reduce_right

    #
    # (4) αチャネルプレーンの準備
    # 画像はみ出しを考慮し、矩形サイスを調節する
    #
    if (y_reduce_upper != 0 and x_reduce_left == 0):	# 昆虫画像がY方向上部にはみ出ており、X方向左にはみ出ていないケース
        alpha_insect = img_insect[y_reduce_upper:insect_height, 0:(x2-x1), 3] / 255.0
    elif (y_reduce_upper == 0 and x_reduce_left != 0):	# 昆虫画像がY方向上部にはみ出ておらず、X方向左にはみ出ているケース
        alpha_insect = img_insect[0:(y2-y1), x_reduce_left:insect_width, 3] / 255.0
    else: # 昆虫画像がY方向上部にも、X方向左にもはみ出ていないケース
        alpha_insect = img_insect[0:(y2-y1), 0:(x2-x1), 3] / 255.0

    alpha_back = 1.0 - alpha_insect

    #
    # (5) BGRプレーン合成処理
    #
    for c in range(0, 3):
        if (y_reduce_upper != 0 and x_reduce_left == 0):	# 昆虫画像がY方向上部にはみ出ており、X方向左にはみ出ていないケース
            img_1 = alpha_insect * img_insect[y_reduce_upper:insect_height, 0:(x2-x1), c]
        elif (y_reduce_upper == 0 and x_reduce_left != 0):	# 昆虫画像がY方向上部にはみ出ておらず、X方向左にはみ出ているケース
            img_1 = alpha_insect * img_insect[0:(y2-y1), x_reduce_left:insect_height, c]
        else: # 昆虫画像がY方向上部にも、X方向左にもはみ出ていないケース
            img_1 = alpha_insect * img_insect[0:(y2-y1), 0:(x2-x1), c]

        img_2 = alpha_back * img_back[y1:y2, x1:x2, c]
        img_back[y1:y2, x1:x2, c] = img_1 + img_2

def place_insect_along_line(back_img, insect_start_point, insect_end_point, idx, draw_figure):

    # 2点を結ぶ直線を描画
    if draw_figure == 1:
        #pass
        cv2.line(back_img, pt1=insect_start_point, pt2=insect_end_point, color=GREEN, thickness=THCK)

    # 楕円を描画 
    mp_x, mp_y = calc_midpoint(insect_start_point, insect_end_point)
    dist = calc_distance(insect_start_point, insect_end_point)
    angle = calc_angle(insect_start_point, insect_end_point)

    if draw_figure == 1:
        pass
        #cv2.ellipse(back_img, (mp_x, mp_y), (dist//4, dist//2), angle, 0, 360, color=CYAN, thickness=THCK)	# 縦長の楕円blue

    # 内側に平行移動した楕円を描画################################試
    delta_x = insect_end_point[0] - insect_start_point[0]
    delta_y = insect_end_point[1] - insect_start_point[1]
    
    direction_x = delta_y / dist
    direction_y = (-1) * delta_x / dist

    mp_x_moved = int(mp_x + direction_x * (dist//4))
    mp_y_moved = int(mp_y + direction_y * (dist//4))
    mp_x_moved_2 = int(mp_x + direction_x * (dist//4)/1.7)
    mp_y_moved_2 = int(mp_y + direction_y * (dist//4)/1.7)
    if draw_figure == 1:
        pass
        #cv2.ellipse(back_img, (mp_x_moved, mp_y_moved), (dist//4, dist//2), angle, 0, 360, color=RED, thickness=THCK)	# 縦長の楕円red
        #cv2.ellipse(back_img, (mp_x_moved_2, mp_y_moved_2), (dist//4, dist//2), angle, 0, 360, color=YELLOW, thickness=THCK)    #yellow
    
    # 昆虫画像のロード
    if USE_LINE_INSECT == 3:
        insect_file_path = './images/wakuta_c2/' + insect_images_AI_c_new[idx % len(insect_images_AI_c_new)] + '_AI.png'
    else:
        pass

    
    insect_img = cv2.imread(insect_file_path, cv2.IMREAD_UNCHANGED)

    # ===============================================================
    # αチャネル付き昆虫画像をアフィン変換(回転/拡縮の実行)
    # ===============================================================
    insect_height, insect_width = (insect_img.shape[0] , insect_img.shape[1])
    scale = dist / insect_height * 1.5	# 画像内の余白部を考慮し、拡大する

    insect_size = (insect_width, insect_height)
    center = (insect_width / 2, insect_height / 2)

    angle_mod = (-angle + 180 + 360) % 360

    # 回転変換行列の算出
    rot_mtx = cv2.getRotationMatrix2D(center, angle_mod, scale)

    # アフィン変換
    insect_img_rot = cv2.warpAffine(insect_img, rot_mtx, insect_size, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

    # 合成処理#############要改善
    process_image(insect_img_rot, back_img, mp_x_moved_2, mp_y_moved_2)



def main():
    # 元画像の読み込み 640×427
    img_back = cv2.imread('images/background/back_wakuta.jpg')

    # 描画用に元画像をコピーする
    copied_img = img_back.copy()

    # 輪郭抽出処理
    img_mask = cv2.imread('images/background/wakuta_mask.png')
    #img_mask = cv2.imread('images/background/back_mask2.png')
    #img_mask = cv2.imread('images/background/horse2_mask.png')
    #img_mask = cv2.imread('images/background/rabbit_mask.png')
    #img_mask = cv2.imread('images/background/cat2_mask.png')
    
    contour_back = get_largest_contour(img_mask)

    points_outer = []
    points2_outer = []

    # 輪郭近似直線群、交点群取得処理
    points_outer = cv2.approxPolyDP(contour_back, 0.003*cv2.arcLength(contour_back, True), True)

    points2_outer = cv2.approxPolyDP(contour_back,0.001*cv2.arcLength(contour_back, True), True)

    # 輪郭直線毎昆虫配置処理#################################################################
    if PLACE_EDGE_INSECTS == 1:

        draw_fig = (1 if (DRAW_EDGE_FIGURE == 1) else 0)	# ガイド図形描画有無

        # 昆虫画像の読み込み
        insect_img = cv2.imread('images/wakuta_c2/mantis4_AI.png', cv2.IMREAD_UNCHANGED)

        
        if insect_img is not None:
            # 昆虫画像を上下に分割
            h, w = insect_img.shape[:2]
            chest_img = insect_img[:h//2, :]  # 胸部
            abdomen_img = insect_img[h//2:, :]  # 腹部

            for k in range(0, len(points_outer) - 2):  # セグメント間の角度を計算するので -2
                    # セグメントの始点と終点
                    start_point1 = [points2_outer[k][0][0], points2_outer[k][0][1]]
                    end_point1 = [points2_outer[k + 1][0][0], points2_outer[k + 1][0][1]]

                    # 次のセグメントの始点と終点
                    start_point2 = end_point1
                    end_point2 = [points2_outer[k + 2][0][0], points2_outer[k + 2][0][1]]

                    # ベクトル計算
                    vec1 = (end_point1[0] - start_point1[0], end_point1[1] - start_point1[1])  # セグメント1のベクトル
                    vec2 = (end_point2[0] - start_point2[0], end_point2[1] - start_point2[1])  # セグメント2のベクトル

                    # 角度計算
                    angle = calculate_angle(vec1, vec2)
                    print(f"擬似輪郭直線2のセグメント {k + 1} と {k + 2} の間の角度: {angle} 度")  
                    #回転
                    rotated_chest = rotate_image(chest_img, angle, (w//2 ,h//2))
                    rotated_abdomen = rotate_image(abdomen_img, 0 )
                            
                    # 合成画像の作成
                    combined_height = rotated_chest.shape[0] + rotated_abdomen.shape[0]
                    combined_width = max(rotated_chest.shape[1], rotated_abdomen.shape[1])
                    combined_img = np.zeros((combined_height, combined_width, 4), dtype=np.uint8)  # 透明領域を含む合成画像

                    # 胸部画像を合成
                    combined_img[:rotated_chest.shape[0], :rotated_chest.shape[1]] = rotated_chest
                    # 腹部画像を合成
                    combined_img[rotated_chest.shape[0]:, :rotated_abdomen.shape[1]] = rotated_abdomen

                    #
                    file_name = f'./images/wakuta_c2/mantis1{k}_AI.png'
                            
                    # 保存
                    cv2.imwrite(file_name, combined_img)
                    print("合成画像が保存されました。")
                    print(k,"\n-----------------------")
            for i in range(0, len(points_outer)):
                insect_start_point = [points_outer[i][0][0], points_outer[i][0][1]]
                j = (i+1) if (i+1) != len(points_outer) else 0
                insect_end_point = [points_outer[j][0][0], points_outer[j][0][1]]

                place_insect_along_line(copied_img, insect_start_point, insect_end_point, i, draw_fig)
              
    cv2.imwrite("./results/results_wakuta/wakuta16_result5.png", copied_img)



if __name__ == "__main__":
    main()

擬似輪郭直線2のセグメント 1 と 2 の間の角度: 22 度
合成画像が保存されました。
0 
-----------------------
擬似輪郭直線2のセグメント 2 と 3 の間の角度: 23 度
合成画像が保存されました。
1 
-----------------------
擬似輪郭直線2のセグメント 3 と 4 の間の角度: 17 度
合成画像が保存されました。
2 
-----------------------
擬似輪郭直線2のセグメント 4 と 5 の間の角度: 19 度
合成画像が保存されました。
3 
-----------------------
擬似輪郭直線2のセグメント 5 と 6 の間の角度: 18 度
合成画像が保存されました。
4 
-----------------------
擬似輪郭直線2のセグメント 6 と 7 の間の角度: 24 度
合成画像が保存されました。
5 
-----------------------
擬似輪郭直線2のセグメント 7 と 8 の間の角度: 20 度
合成画像が保存されました。
6 
-----------------------
擬似輪郭直線2のセグメント 8 と 9 の間の角度: 14 度
合成画像が保存されました。
7 
-----------------------
擬似輪郭直線2のセグメント 9 と 10 の間の角度: 40 度
合成画像が保存されました。
8 
-----------------------
擬似輪郭直線2のセグメント 10 と 11 の間の角度: 11 度
合成画像が保存されました。
9 
-----------------------
擬似輪郭直線2のセグメント 11 と 12 の間の角度: 16 度
合成画像が保存されました。
10 
-----------------------
擬似輪郭直線2のセグメント 12 と 13 の間の角度: 22 度
合成画像が保存されました。
11 
-----------------------
擬似輪郭直線2のセグメント 13 と 14 の間の角度: 21 度
合成画像が保存されました。
12 
-----------------------
擬似輪郭直線2のセグメント 14 と 15 の間の角度: 11 度
合成画像が